In [ ]:
import json

def dumper(obj):
    try:
        return obj.toJSON()
    except:
        return obj.__dict__

class index_info(object):
        
    def __init__(self, index):
        self.index = index 
        self.boundaries = []
        self.layers_no_approach = []
        self.layers_with_approach = []
        self.approaches = []
        self.passiveDiscBinningR = 0
        self.passiveDiscBinningPhi = 0
        self.passiveCylinderBinningZ = 0
        self.passiveCylinderBinningPhi = 0
        self.activeBinningRorZ = 0
        self.activeBinningPhi = 0
    
    def __repr__(self):
        return repr((self.index, self.boundaries, self.layers_no_approach, 
                     self.layers_with_approach, self.approaches,
                     self.passiveDiscBinningR, self.passiveDiscBinningPhi, 
                     self.passiveCylinderBinningZ, self.passiveCylinderBinningPhi, 
                     self.activeBinningRorZ, self.activeBinningPhi))   
    
def dump_geo(f, min_x = -1, max_x = -1, min_y = -1, max_y = -1):
    data = json.load(f)

    index_to_extends_volumes = []
    index_to_names = []
    central_extensions = {}
    
    plotLayerIndex=True

    # First you want to read the Volume entries and save the relevant quantities
    # - names, extensions, binnings
    
    max_value = -1
    for entry in data['Volumes']['entries']:
        index_to_names.append(entry['value']['NAME'])
        extends = []
        for val in entry['value']['material']['binUtility']['binningdata']:
            if val['value'] == 'binR' or  val['value'] == 'binZ':
                #print (entry['volume'], val['value'], val['min'], val['max'])
                extends.append(val['min'])
                extends.append(val['max'])
            if val['value'] == 'binZ' and "|" not in entry['value']['NAME'] and 'Barrel' in entry['value']['NAME']:
                central_extensions[(entry['value']['NAME']).split(':')[0]] = val['max']
                if val['max']>max_value:
                    max_value = val['max']
        index_to_extends_volumes.append(extends)
    
    # Check if all expected volumes have extends, otherwise evaluate it
    for name in index_to_names:
        if "|" not in name and name.split(':')[0] not in central_extensions and max_value>0.:
            central_extensions[name.split(':')[0]] = max_value
            
    
            
    # Stores the information to easily decide on which layers you want the material to be mapped
    steering_map = {}

    # Once you have read the volumes extensions, you read the surfaces representing layers and boundaries.
    # All these surfaces belong to a volume, they have therefore a volume index.
    # You are interested only in:
    # - surfaces with layer index (NO approach index): 
    #    -- even layer indices represent "active" layers, i.e. the ones corresponding to sensitive layers 
    #    -- odd event indices represent navigation layers
    # - surfaces with layer index AND approach index:
    #    -- represent approach layers to "active" layers: 
    #       e.g. it can happen that for cylinders: 1 sits in front of the active layer, 
    #                                              2 sits in the middle of the layer, 
    #                                              3 sits behind the layer.
    # - surfaces with boundary index (no layer index in this case):
    #    -- they represent boundaries between volumes. You are interested in boundaries between volumes 
    #       containing at least an active layer.

    index_to_extends_layers_approach_cylinders = [[] for _ in range(len(index_to_extends_volumes))]
    index_to_extends_layers_approach_discs = [[] for _ in range(len(index_to_extends_volumes))]

    index_to_extends_layers_bounds_cylinders = [[] for _ in range(len(index_to_extends_volumes))]
    index_to_extends_layers_bounds_discs = [[] for _ in range(len(index_to_extends_volumes))]

    index_to_extends_layers_approach_with_material = [[] for _ in range(len(index_to_extends_volumes))]

    index_to_extends_layers_cylinders = [[] for _ in range(len(index_to_extends_volumes))]
    index_to_extends_layers_discs = [[] for _ in range(len(index_to_extends_volumes))]

    volume_shift = [0] * len(index_to_extends_volumes)

    for entry in data['Surfaces']['entries']:
        if "layer" in entry:
            extends = []
            vol = entry['volume']
            
            if "sensitive" in entry:
                continue

            if entry['value']['type'] == "CylinderSurface":            
                z_shift = 0.
                if entry['value']['transform']['translation'] != None:
                    z_shift = entry['value']['transform']['translation'][2]

                extends = [0., entry['value']['bounds']['values'][0], 
                           z_shift-entry['value']['bounds']['values'][1], z_shift+entry['value']['bounds']['values'][1]]    

                if "approach" in entry:
                    extends.append(entry["approach"])
                else:
                    extends.append(-1)
                extends.append(entry["layer"])
                    
                if "approach" in entry:
                    index_to_extends_layers_approach_cylinders[vol-1].append(extends)
                else:
                    index_to_extends_layers_cylinders[vol-1].append(extends)


            else:
                z_shift = 0.
                if entry['value']['transform']['translation'] != None:
                    z_shift = entry['value']['transform']['translation'][2]

                extends = [entry['value']['bounds']['values'][0], entry['value']['bounds']['values'][1],z_shift]    

                if "approach" in entry:
                    extends.append(entry["approach"])
                else:
                    extends.append(-1)
                extends.append(entry["layer"])
                    
                if "approach" in entry:
                    index_to_extends_layers_approach_discs[vol-1].append(extends)
                else:
                    index_to_extends_layers_discs[vol-1].append(extends)

                    if "|" not in index_to_names[vol-1] and 'Endcap' in index_to_names[vol-1]:
                        if index_to_names[vol-1].split(':')[0] in central_extensions:
                            volume_shift[vol-1] = central_extensions[index_to_names[vol-1].split(':')[0]]
                            if 'Negative' in index_to_names[vol-1]:
                                volume_shift[vol-1] = -1.*volume_shift[vol-1]
                        else:
                            volume_shift[vol-1] = 0

        if "boundary" in entry:
            extends = []
            vol = entry['volume']

            if entry['value']['type'] == "CylinderSurface":            
                z_shift = 0.
                if entry['value']['transform']['translation'] != None:
                    z_shift = entry['value']['transform']['translation'][2]

                extends = [0., entry['value']['bounds']['values'][0], 
                           z_shift-entry['value']['bounds']['values'][1], z_shift+entry['value']['bounds']['values'][1],entry['boundary']]    

                index_to_extends_layers_bounds_cylinders[vol-1].append(extends)

            else:
                z_shift = 0.
                if entry['value']['transform']['translation'] != None:
                    z_shift = entry['value']['transform']['translation'][2]

                extends = [entry['value']['bounds']['values'][0], entry['value']['bounds']['values'][1],z_shift,entry['boundary']]    

                index_to_extends_layers_bounds_discs[vol-1].append(extends)
                
    # Once you have all the data in hands, you make a few nice plots to show volumes and surfaces

    import matplotlib.pyplot as plt
    from matplotlib.pyplot import cm
    from itertools import cycle
    import numpy as np

    plt.figure(figsize=(20,10))

    color = cm.rainbow(np.linspace(0, 1, len(index_to_extends_layers_cylinders)))
    v_index = 0
    l_index = 0

    interesting_volumes = []
    
    is_in_legend = []
    
    for elements in index_to_extends_layers_cylinders:
        l_index = 0
        for coords in elements:
            if index_to_names[v_index] not in is_in_legend:
                is_in_legend.append(index_to_names[v_index])
            l_index = l_index+1
            if v_index not in interesting_volumes:
                interesting_volumes.append(v_index)
            
            if index_to_names[v_index] not in steering_map:
                steering_map[index_to_names[v_index]] = index_info(v_index+1)
                
            steering_map[index_to_names[v_index]].layers_no_approach.append(coords[5])
            
        v_index = v_index+1


    v_index=0
    for elements in index_to_extends_layers_discs:
        l_index = 0
        for coords in elements:
            if index_to_names[v_index] not in is_in_legend:
                is_in_legend.append(index_to_names[v_index])
            l_index = l_index+1
            if v_index not in interesting_volumes:
                interesting_volumes.append(v_index)
                
            if index_to_names[v_index] not in steering_map:
                steering_map[index_to_names[v_index]] = index_info(v_index+1)
                
            steering_map[index_to_names[v_index]].layers_no_approach.append(coords[4])            
                
        v_index = v_index+1   

    v_index=0
    for elements in index_to_extends_layers_approach_cylinders:
        l_index = 0
        for coords in elements:
            C, D = [coords[3], coords[1]], [coords[2], coords[1]]

            x_values = [D[0], C[0]]
            y_values = [D[1], C[1]]

            if coords[4]==2:
                plt.plot(x_values, y_values, c=color[v_index])
        v_index = v_index+1   

    v_index=0
    for elements in index_to_extends_layers_approach_discs:
        l_index = 0
        for coords in elements:
            C,D = [coords[2], coords[0]], [coords[2], coords[1]]

            x_values = [D[0], C[0]]
            y_values = [D[1], C[1]]
            if coords[3]==2:
                plt.plot(x_values, y_values, c=color[v_index])          
                
        v_index = v_index+1   


    v_index = 0
    for elements in index_to_extends_layers_bounds_cylinders:
        l_index = 0
        for coords in elements:
            C, D = [coords[3], coords[1]], [coords[2], coords[1]]

            x_values = [D[0], C[0]]
            y_values = [D[1], C[1]]

            if v_index in interesting_volumes:
                plt.plot(x_values, y_values, c=color[v_index])
#                 , label='v: '+str(v_index+1)+', '+index_to_names[v_index]
                
            l_index = l_index+1
        v_index = v_index+1

    v_index=0
    for elements in index_to_extends_layers_bounds_discs:
        l_index = 0
        for coords in elements:
            C,D = [coords[2], coords[0]], [coords[2], coords[1]]

            x_values = [D[0], C[0]]
            y_values = [D[1], C[1]]

            if v_index in interesting_volumes:
                plt.plot(x_values, y_values, c=color[v_index])
                
            l_index = l_index+1
        v_index = v_index+1 
        

    points = []    
    
       
    
    from math import sqrt
    x_values = []
    y_values = []
    z_txt = []
    l_txt = []
    el = 0
    for point in points:
        x_values.append (point[2])
        y_values.append (sqrt(point[1]*point[1]+point[0]*point[0])   )
        if (min_x!=-1 and max_x!=-1 and x_values[len(x_values)-1]<max_x and x_values[len(x_values)-1]>min_x \
            and min_y!=-1 and max_y!=-1 and y_values[len(y_values)-1]<max_y and y_values[len(y_values)-1]>min_y):
            offset = 0
            if y_values[len(y_values)-1]<300:
                counts_z = z_txt.count(x_values[len(x_values)-1])
                z_txt.append(x_values[len(x_values)-1])
                counts_l = 0
                if counts_z>1:
                    counts_l = l_txt.count(point[3])
                    l_txt.append(point[3])
                offset = 2.+counts_l*5.+counts_z*5
            else:
                counts_l = l_txt.count(point[3])
                l_txt.append(point[3])
                offset = 10.+counts_l*10.
            index = el
            if len(point)==5:
                index = point[4]
            plt.text(x_values[len(x_values)-1], y_values[len(y_values)-1]+offset, str(index))
        if len(point)<5:
            el=el+1
    plt.plot(x_values, y_values, linestyle='-', c= "blue", marker="o")
    
    x_values = []
    y_values = []
    
    points = [ 
              [34.4364, 139.6 ],
              [88.5289, 359],
              [98.15, 398],
              [108.251, 439],
              [120.333, 488 ],
              [173.746, 704.277],
              [223.837, 906.981],
              [225.869, 915.216],
              [282.266, 1143],
              [314.81, 1274.5],]

    R_values = [33.273, 97.150, 157.479, 408.819, 387.840, 394.342, 554.994, 561.693, 768.641, 775.688, 756.795, 990.084, 996.750]
    Z_values = [28.933, 73.575, 116.075, 282.583, 268.045, 282.596, 392.199, 405.286, 540.184, 562.120, 564.513, 685.703, 711.878]
    points = list(zip(R_values, Z_values))
    
#     points = []

    for point in points:        
        x_values.append (point[1])
        y_values.append (point[0]) 
    plt.plot(x_values, y_values, linestyle='-', markerfacecolor='green', c= "green", marker="o")
    
    points = [ 
               [34.436,139.518],
               [87.5446, 355],
               [97.1813, 394],
               [107.311, 435],
               [119.439, 484 ],
               [172.113, 696.421],
               [224.315, 906.715],
               [226.335, 914.884],
               [254.704, 1029.06],
               [281.592, 1137],
               [314.363, 1268.5]]

#     points=[]
    x_values=[]
    y_values=[]
    for point in points:        
        x_values.append (point[1])
        y_values.append (point[0]) 
#     plt.plot(x_values, y_values, linestyle=(0, (1, 10)), c= "blue", markerfacecolor='none', marker="o")

    elements = [[34 , 131.95],[34 , 152.25],[34 , 111.65],[34 , 213.15],[34 , 192.85],[34 , 172.55],[42.8 , 261],[42.8 , 261],[100.1 , 261],[100.1 , 261],[42.8 , 265],[100.1 , 265],[100.1 , 289],[100.1 , 289],[42.8 , 289],[42.8 , 289],[100.1 , 293],[42.8 , 293],[100.1 , 320],[100.1 , 320],[100.1 , 324],[100.1 , 355],[100.1 , 355],[100.1 , 359],[100.1 , 398],[100.1 , 439],[100.1 , 488],[166.231 , 577.568],[166.231 , 698.768],[170.705 , 705.4],[232.777 , 901.072],[237.251 , 907.704],[232.777 , 1032.97],[237.251 , 1039.6],[294.7 , 1143],[294.7 , 1274.5],[393.99 , 1516.29],[393.99 , 1522.71],[415.471 , 1691.29],[393.99 , 1691.29],[441.952 , 1691.29],[441.952 , 1697.71],[393.99 , 1697.71],[415.471 , 1706.29],[441.952 , 1706.29],[393.99 , 1706.29],[415.471 , 1712.71],[441.952 , 1712.71],[393.99 , 1712.71],[498.869 , 1941.29],[498.869 , 1947.71],[472.432 , 1956.29],[498.869 , 1956.29],[472.432 , 1962.71],[498.869 , 1962.71],[566.647 , 2226.28],[547.055 , 2241.28],[566.647 , 2241.28],[521.462 , 2241.28],[547.055 , 2247.72],[566.647 , 2247.72],[521.462 , 2247.72],[654.72 , 2521.28],[621.805 , 2536.28],[654.72 , 2536.28],[590.998 , 2536.28],[621.805 , 2542.72],[654.72 , 2542.72],[590.998 , 2542.72],[710.166 , 2854.28],[683.943 , 2854.28],[710.166 , 2860.72],]
    
    x_values=[]
    y_values=[]
    for element in elements:
        x_values.append (element[1])
        y_values.append (element[0]) 
#     plt.plot(x_values, y_values, linestyle=(0, (1, 10)), c= "blue", markerfacecolor='none', marker="o")

    
    
    seeds = [[[ 88.5289, 359], [ 98.15, 398], [ 225.869, 915.216]]]
    
#     seeds = [[[ 88.5289, 359], [ 120.333, 488], [ 223.837, 906.981]]]
    
#     seeds = [[[ 88.5289, 359], [ 173.746, 704.277], [ 282.266, 1143]]]
    
#     seeds = [[[ 88.5289, 359], [ 173.746, 704.277], [ 314.81, 1274.5]]]
    
#     seeds = [[[ 98.15, 398], [ 173.746, 704.277], [ 282.266, 1143]]]
    
             
    import numpy as np
    for seed in seeds:
        x_values = []
        y_values = []
        for point in seed:        
            x_values.append (point[1])
            y_values.append (point[0]) 
#         plt.plot(x_values, y_values, linestyle='--', c = (np.random.random(), np.random.random(), np.random.random()), marker="o")
        plt.plot(x_values, y_values, linestyle='--', c = "red", marker="o")

        
    seeds = [[[ 108.251, 439], [ 173.746, 704.277], [ 282.266, 1143]]]
             
    
    for seed in seeds:
        x_values = []
        y_values = []
        for point in seed:        
            x_values.append (point[1])
            y_values.append (point[0]) 
        plt.plot(x_values, y_values, linestyle=(0, (1, 10)), c= "blue", markerfacecolor='none', marker="o")
    
    if (min_x!=-1 and max_x!=-1):
        plt.xlim(min_x, max_x)
        
    if (min_y!=-1 and max_y!=-1):
        plt.ylim(min_y, max_y)
        
    lines = [-2500, -1400, -925, -500, -250,250, 500, 925, 1400, 2500]
    x_values = []
    y_values = []
    
    for p in lines:
        y_values=[0, 1000]
        x_values=[p, p]
        plt.plot(x_values, y_values, c= "red", marker="o")
        
    lines = [40, 70, 123.]
    x_values = []
    y_values = []
    
    for p in lines:
        x_values=[-3000, 3000]
        y_values=[p, p]
        plt.plot(x_values, y_values, c= "red", marker="o")
        
    lines = [200., 250.]
    x_values = []
    y_values = []
    
    for p in lines:
        x_values=[-3000, 3000]
        y_values=[p, p]
        plt.plot(x_values, y_values, c= "blue", marker="o")
        
    plt.xlabel('z [mm]')
    plt.ylabel('R [mm]')
    plt.title("Volumes and Layers")
    plt.show()

    

In [ ]:
#f = open('/home/noemi/Work/athena/run-acts-mapping/geometry-maps-ATLAS-P2-RUN4-01-00-00.json')
#f = open('../test_athena.json')
f = open('geometry-maps-ATLAS-P2-RUN4-03-00-00.json')
dump_geo(f)

In [ ]:
f = open('../test_athena.json')
dump_geo(f, 0, 1500, 0, 150)

In [ ]:
f = open('../test_athena.json')
dump_geo(f, 250, 800, 300, 1050)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from itertools import cycle
import numpy as np

plt.figure(figsize=(20,10))

x_values = []
y_values = []


points = [[23.0656, -23.6059 , -42.8833 ], 
          [26.2041, -26.8057 , -55.55   ], 
          [68.0036, -68.3346 , -222.7   ], 
          [73.6459, -73.8484 , -244.975 ], 
          [77.6934, -77.7585 , -261     ], 
          [78.7093, -78.7241 , -265     ], 
          [84.785, -84.627 , -289       ], 
          [85.8147, -85.617 , -293      ], 
          [116.042, -114.339 , -411.311 ], 
          [133.682, -130.741 , -479.694 ], 
          [ 165.074, -159.387 , -600.551], 
          [ 220.599, -208.34 , -811.345 ], 
          [294.915, -270.365 , -1093.96 ], 
          [412.175, -363.157 , -1501.29 ], 
          [458.34, -397.049 , -1691.29  ], 
          [463.454, -401.302 , -1691.29 ], 
          [535.714, -453.623 , -1941.29 ], 
          [614.845, -508.518 , -2241.28 ], 
          [618.16, -511.52 , -2226.28   ], ]

for point in points:        
    x_values.append (point[0])
    y_values.append (point[1]) 
plt.plot(x_values, y_values, linestyle='-', c= "green", marker="o")
